In [9]:
import json
import re
from datetime import datetime

import mysql.connector
import requests
from mysql.connector import IntegrityError


In [66]:
from mysql.connector import DataError


def insert_course(course_info: dict, course_json: dict):
    if course_info is None: return

    connection = mysql.connector.connect(
        host="coms-309-029.class.las.iastate.edu",
        port=3306,
        user="root",
        password="5b36fb66f5d37314",
        database="Scheduler"
    )
    cursor = connection.cursor()

    sql = (
        "INSERT INTO Course (program_identifier, num, display_name, description, credits, is_variable_credit, is_graded)" +
        "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    )
    values = (
        course_info["program_identifier"],
        course_info["num"],
        course_info["displayName"],
        course_info["description"].encode("latin-1").decode("utf-8"),
        course_info["credits"],
        course_info["is_variable_credit"],
        course_info["is_graded"]
    )
    try:
        cursor.execute(sql, values)
        connection.commit()
    except IntegrityError:
        print(f"Duplicate entry: {course_info['program_identifier']} {course_info['num']}")

    year = 2024
    season = 1
    for sec in get_sections(course_json):
        try:
            cursor.callproc("addSection", (
                sec["ref_num"],
                course_info["program_identifier"],
                course_info["num"],
                sec["section"],
                year,
                season,
                sec["is_online"],
            ))
            connection.commit()
        except IntegrityError:
            print(f"Duplicate entry: {sec['section']}")
        except DataError:
            print(f"bad data")

        for t in sec["schedule"]:
            try:
                cursor.callproc("addSectionSchedule", (
                    course_info["program_identifier"],
                    course_info["num"],
                    sec["section"],
                    t["start_time"],
                    t["end_time"],
                    t["meet_days_bitmask"],
                    t["location"],
                    t["instructor"],
                    t["instruction_type"]
                ))
                connection.commit()
            except IntegrityError:
                print(f"Duplicate entry: {sec['section']}")

    connection.commit()
    cursor.close()
    connection.close()

In [56]:
def scrape_course(program: str, num: int, sem: int = 3) -> tuple[dict, dict] | None:
    url = "https://classes.iastate.edu/app/rest/courses/preferences"
    headers: dict = {"Content-Type": "application/json"}
    payload: str = json.dumps({
        "defSem": sem,
        "selectedTerm": sem,
        "selectedDepartment": program,
        "courseNumber": num
    })

    response: dict = requests.post(url, headers=headers, data=payload).json()
    if len(response["response"]) == 0: return None
    course_data: dict = response["response"][0]
    catalog_data: str = requests.get(course_data["catalogUrl"]).text

    # Don't want experimental courses
    if course_data["classNumber"][-1] == "X": return None
    try:
        return {
            "program_identifier": course_data["deptCode"],
            "num": int(re.search(r"([0-9]{1,3})[a-zA-Z]{,3}", course_data["classNumber"]).group(1)),
            "displayName": re.search(r"&#160;[0-9]{1,3}[A-Za-z]{,3}:(.+)</strong>", catalog_data, re.DOTALL).group(
                1).strip() if len(catalog_data) > 55 else None,
            "description": re.search(r"><br />(.+)</p>", catalog_data, re.DOTALL).group(1).strip() if len(
                catalog_data) > 55 else None,
            "credits": int(course_data["creditLow"]),
            "is_variable_credit": course_data["creditType"] == 'V',
            "is_graded": course_data["gradeType"] == ""
        }, course_data
    except AttributeError:
        return None


def meet_days_to_bitmask(s: str):
    mask = 0
    for c in s:
        if c == "M":
            mask |= 1
        elif c == "T":
            mask |= 2
        elif c == "W":
            mask |= 4
        elif c == "R":
            mask |= 8
        elif c == "F":
            mask |= 16
    return mask


def convert_time(s: str):
    t = datetime.strptime(s, "%H:%M:%S.%f")
    return t.hour * 60 + t.minute


def get_sections(course_json: dict) -> list[dict]:
    sections: list[dict] = []
    for section_data in course_json["sections"]:
        is_online = None if section_data["deliveryTypeDisplay"] == "" else "online" in section_data[
            "deliveryTypeDisplay"]

        times = []
        for section_time in section_data["sectionTimes"]:
            location = f'{section_time["buildingName"]} {section_time["roomNum"]}'.strip()
            section = section_data["sectionID"].strip()
            instructor = section_time["instrName"].strip()
            instruction_type = section_time["instructionType"].strip()
            meet_days = section_time["meetDays"].strip()

            times.append({
                "start_time": None if is_online else convert_time(section_time["startTime"]),
                "end_time": None if is_online else convert_time(section_time["stopTime"]),
                "meet_days_bitmask": None if meet_days == "" else meet_days_to_bitmask(meet_days),
                "location": None if location == "" else location,
                "instructor": None if instructor == "" else instructor,
                "instruction_type": None if instruction_type == "" else instruction_type
            })

        sections.append({
            "section": None if section == "" else section,
            "ref_num": section_data["referenceNumber"],
            "is_online": is_online,
            "schedule": times
        })

    return sections

In [63]:
insert_course(*scrape_course("MATH", 166))

Duplicate entry: MATH 166


In [70]:
depts = ["CPR E", "S E", "STAT", "ENGL", "E E", "M E", "ECON", "PHIL"]
for dept in depts:
    for n in range(100, 700):
        try:
            c = scrape_course(dept, n, sem=3)
        
            if c is None:
                print(f"{dept} {n} doesn't exist")
                continue
            print(f"Inserting {dept} {n}")
        
            insert_course(*c)
        except Exception:
            pass


CPR E 100 doesn't exist
CPR E 101 doesn't exist
CPR E 102 doesn't exist
CPR E 103 doesn't exist
CPR E 104 doesn't exist
CPR E 105 doesn't exist
CPR E 106 doesn't exist
CPR E 107 doesn't exist
CPR E 108 doesn't exist
CPR E 109 doesn't exist
CPR E 110 doesn't exist
CPR E 111 doesn't exist
CPR E 112 doesn't exist
CPR E 113 doesn't exist
CPR E 114 doesn't exist
CPR E 115 doesn't exist
CPR E 116 doesn't exist
CPR E 117 doesn't exist
CPR E 118 doesn't exist
CPR E 119 doesn't exist
CPR E 120 doesn't exist
CPR E 121 doesn't exist
CPR E 122 doesn't exist
CPR E 123 doesn't exist
CPR E 124 doesn't exist
CPR E 125 doesn't exist
CPR E 126 doesn't exist
CPR E 127 doesn't exist
CPR E 128 doesn't exist
CPR E 129 doesn't exist
CPR E 130 doesn't exist
CPR E 131 doesn't exist
CPR E 132 doesn't exist
CPR E 133 doesn't exist
CPR E 134 doesn't exist
CPR E 135 doesn't exist
CPR E 136 doesn't exist
CPR E 137 doesn't exist
CPR E 138 doesn't exist
CPR E 139 doesn't exist
CPR E 140 doesn't exist
CPR E 141 doesn'

In [ ]:
(scrape_course("MATH", 699)[0]["description"].encode("latin-1").decode("utf-8"))

In [330]:
re.search(r"&#160;[0-9]{1,3}[A-Za-z]{,3}:(.+)</strong>",
          requests.get("https://catalog.iastate.edu/ribbit/?page=getcourse.rjs&code=MATH%20699C&edition=2023-24").text,
          re.DOTALL)

In [14]:
requests.get("https://catalog.iastate.edu/ribbit/?page=getcourse.rjs&code=CPR E%20529&edition=2023-24").text.encode(
    "latin-1").decode("utf-8")

'<?xml version="1.0"?>\n<courseinfo>\n<course code="CPR E 529">\n<![CDATA[\n<div class="courseblock">\n<div class=\'courseblocktitle\'><a href=\'#\' class=\'toggle-accordion courseblocklink\'><strong>CPR&#160;E&#160;529: Data Analytics in Electrical and Computer Engineering</strong><span></span></a></div><div class="courseblockdesc accordion-content"><p class="credits noindent">\n(Cross-listed with E E).  (3-0) Cr. 3.\n S. \n</p><p class=\'prereq\'><em>Prereq: E E 322 or equivalent</em><br />Introduces a variety of data analytics techniques ‐ particularly those relevant for electrical and computer engineers ‐ from a foundational perspective. Topics to be covered include techniques for classification, visualization, and parameter estimation, with applications to signals, images, matrices, and graphs. Emphasis will be placed on rigorous analysis as well as principled design of such techniques.\n</p></div>\n</div>\n]]>\n</course>\n</courseinfo>\n'